# Module 8 : Classifying Iris with Neural Networks

In this session, you will create a Multilayer Perceptron (MLP) model to practice on the Iris dataset, to get more familiarized with PyTorch.



#### Some initial library loading!

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import os, sys
import itertools
import numpy as np
import pandas as pd
from sklearn.preprocessing import scale, LabelEncoder, StandardScaler, Normalizer, MinMaxScaler
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from scipy import stats

import torch
from torch import nn
from torch import optim
from torch.utils.data import TensorDataset, DataLoader
from torch.autograd import Variable
import torch.nn.functional as F
import tqdm

# Random seed for numpy
np.random.seed(18937)

ModuleNotFoundError: No module named 'torch'

#### Loading the Data Set

In [ ]:
from sklearn import datasets

iris = datasets.load_iris()
iris.keys()

In [ ]:
# Features in this dataset
print('features', iris.feature_names)

# Target classes of this 3-class prediction problem.
print('targets', iris.target_names)

In [ ]:
df = pd.DataFrame(iris.data, columns = iris.feature_names)
df.head()

#### Task 1: Assign features and the target to variables X and y

In [ ]:
# Add code below this comment  (Question #P01)
# ----------------------------------

X = iris.data
y = iris.target



## Do a little more inspection

Is `X` a pandas dataframe or a numpy array? What kind of data does it contain? Let's run some descriptive statistics.

In [ ]:
print('type', type(X))
print('shape', X.shape)
df.describe()

In [ ]:
# Prettify to make that more readable.

for stat, val in stats.describe(X)._asdict().items():
    if stat!='minmax':
        print('{:<10}: {}'.format(stat, val))
    else:
        print('{:<10}: {}'.format('min', val[0]))
        print('{:<10}: {}'.format('max', val[1]))

In [ ]:
from collections import Counter
print('type: ', type(y))
print('shape: ', y.shape)
print(Counter(y))

# Visualize the Data 

We see above that the data features are 4-D.
Generate two visualizations of the data using Red, Blue, Green for the colors of
`setosa`, `versicolor`, `virginica`, respectively.

#### Task 2: First, visualize along dimensions 0,1.


In [ ]:
# Add code below this comment  (Question #P02)
# ----------------------------------

#'sepal length (cm)', 'sepal width (cm)'

x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5

# Plot the training points
plt.scatter(X[:, 0], X[:, 1], c=y, cmap="brg",
            edgecolor='k')
plt.xlabel('Sepal length')
plt.ylabel('Sepal width')

plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)
plt.xticks(())
plt.yticks(())



#### Task 3: Second, visualize along dimensions 2,3

In [ ]:
# Add code below this comment  (Question #P03)
# ----------------------------------

# 'petal length (cm)', 'petal width (cm)'


x_min, x_max = X[:, 2].min() - .5, X[:, 2].max() + .5
y_min, y_max = X[:, 3].min() - .5, X[:, 3].max() + .5

# Plot the training points
plt.scatter(X[:, 2], X[:, 3], c=y, cmap="brg",
            edgecolor='k')
plt.xlabel('Petal length')
plt.ylabel('Petal width')

plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)
plt.xticks(())
plt.yticks(())


### Build am NN model

#### Task 4: Define the neural architecture with PyTorch with the following config

1. A hidden layer with 4 neurons
2. Use sigmoid activation fuction for the hidden layer
3. Convert the output layer values to probabilities


In [ ]:
class MyModel(nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        D_in: number of input
        """
        super(MyModel, self).__init__()
        self.layer1 = nn.Linear(D_in, H) # input to hidden layer
        self.layer2 = nn.Linear(H, D_out) # hidden layer to output layer
        
    def forward(self, x):
        h_pred = torch.sigmoid(self.layer1(x)) # activation function after the first hidden layer. Change it to nn.ReLU to see the difference     
        y_pred = torch.softmax(self.layer2(h_pred), dim=1) # softmax function: converts numbers to probabilities 
        return y_pred



In [ ]:
model = MyModel(X.shape[1], 4, 3)

In [ ]:
!pip install torchsummary

In [ ]:
from torchsummary import summary
summary(model, (X.shape[1],))

#### Task 5

Compile the model with the `Cross Entropy` loss function and `rmsprop` optimizer.
 
 

In [ ]:
# Add code below this comment  (Question #P05)
# ----------------------------------

loss = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(model.parameters(), lr=0.001)  


### Divide the Data into Train and Test splits

To make this this more fun, we are going to use a 70%/30% split on the training and testing data.

#### Task 6: Prepare the data for training the model.

In [ ]:
# Add code below this comment  (Question #P06)
# ----------------------------------

# scale the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(iris.data)

# train/test split 

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3)

# convert train/test to tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float)
X_test_tensor = torch.tensor(X_test, dtype=torch.float)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).view(-1, 1)
y_test_tensor = torch.tensor(y_test, dtype=torch.long).view(-1, 1)

# convert y to one hot encoding: not required if we use cross entropy
# y_train_one_hot = nn.functional.one_hot(y_train_tensor)
# y_test_one_hot = nn.functional.one_hot(y_test_tensor)


In [ ]:
BATCH_SIZE = 1  # it is possible to feed more than one istances to the model. 
# These set of instances is called batch. For simplicity, let's keep one instance per batch

train_data = TensorDataset(X_train_tensor, y_train_tensor)
test_data = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=1)

### Task 7: Train the model on the train data

Choose an appropriate number of epochs and batch size.


In [ ]:
# Add code below this comment  (Question #P07)
# ----------------------------------

N_EPOCHS = 100  # In each epoch, the model iterate over all the instances 

for epoch in range(N_EPOCHS):
    epoch_loss = 0
    epoch_acc = 0
    for x, y in train_loader:
        y_pred = model(x)      # Forward pass: get the network output for this instance
        l = loss(y_pred, y.view(-1))    # estimate error for this instance
        epoch_loss += l.item() # Aggregate error
        
        optimizer.zero_grad()  # As backward method accumulates gradient, we need to set it to 0
        l.backward()           # Backward pass: Estimate gradient 
        optimizer.step()
        
        correct = (torch.argmax(y_pred, dim=1) == y).type(torch.FloatTensor)
        epoch_acc += correct.item()

    if (epoch%5)==0:
        print(f'Epoch {epoch+0:03}: | Total Loss: {epoch_loss:.5f} | \
Avg Loss: {epoch_loss/len(train_loader):.5f} | Training Acc: {epoch_acc/len(train_loader):.2f}')

----

### Task 8: Evaluate your trained model on the on the test data

Print the loss and accuracy obtained using model.evaluate(...).

In [ ]:
# Add code below this comment  (Question #P08)
# ----------------------------------

model.eval()

with torch.no_grad():
    y_pred = model(X_test_tensor)


### Task 9: Interpret the classifier output

Run the cell below, then provide your interpretation of the output in the cell below that.  


In [ ]:
print(y_pred.shape)
print(y_pred)

### Task 10: Print the predicted Class Label, then generate a confusion matrix and flassification report
 * Review the documentation: https://docs.scipy.org/doc/numpy/reference/generated/numpy.argmax.html

In [ ]:
# Fix code below this comment  (Question #P10)
# ----------------------------------

from sklearn.metrics import classification_report, confusion_matrix

pred_class = np.argmax(y_pred, axis=1)
print(pred_class)



print(confusion_matrix(y_test, pred_class))
print(classification_report(y_test, pred_class))

---

## This notebook should include things that you are adding to your toolchest in terms of using Scikit-Learn and PyTorch

---

# Save your notebook, then `File > Close and Halt`